<a href="https://colab.research.google.com/github/fhfmendes04/gemini/blob/main/Projeto_Alura_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas Utilizadas

In [1]:
#Instalando
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [2]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [3]:
!pip install python-dotenv

In [4]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.8/677.8 kB 10.6 MB/s eta 0:00:00


In [5]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [6]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.2 MB/s eta 0:00:00


In [7]:
!pip install langchain_google_genai

# APP

In [31]:
%%writefile app.py

import streamlit as st
import google.generativeai as genai
from io import StringIO
import os
import PyPDF2
from PIL import Image
import google.ai.generativelanguage as glm
#from google.colab import userdata
import warnings
warnings.filterwarnings('ignore')

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate


st.set_page_config(page_title="Alura_Gemini", page_icon="📖", layout="wide")
st.header("📖Alura_Gemini")

#api_key = st.sidebar.text_input('GEMINI API Key')


# Deve-se cria um arquivo .env com: API_KEY="seu api_key"
from dotenv import load_dotenv
load_dotenv(".env")

api_key = os.getenv("API_KEY")

genai.configure(api_key=api_key)

os.environ['GOOGLE_API_KEY'] = api_key

# Função para criar o GEMINI
def get_conteudo(messages, model="gemini-pro"):
  model = genai.GenerativeModel(model)
  resultado = model.generate_content(messages)
  return resultado.text

# Função Ler o PDF e gerar o texto
def get_pdf(arquivo):
    pdf_reader = PyPDF2.PdfReader(arquivo)
    # Extract the content
    content = ""
    for page in range(len(pdf_reader.pages)):
        content += pdf_reader.pages[page].extract_text()
    return content

def clear_form():
    st.session_state["entrada"] = ""
    st.session_state["texto"] = ""
    st.session_state["arquivo"] = None
    st.session_state["imagem"] = None
    st.session_state["selecionar"] = None
    st.session_state["enviar"] = None
    st.session_state["resultado"] = None

def cria_vetor(texto):
    """Cria o vetor com o texto com o  embedding do Google Generative AI"""

    # Divide o texto em chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    text_chunks = text_splitter.split_text(texto)

    # Cria um embbedding
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    # Cria um vector store com FAISS
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)

    #Salva um local index
    vector_store.save_local("faiss_index")

def prompt_chain():
    """Cria o prompt e o chain
    Retorna: o chain
    """
    # Cria o prompt
    prompt_template = """
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

    # Cria o promt com a pergunta
    prompt = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    # Cria o chain
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

def conversando(user_question,leitura):
    """Conversando com o PDF
    Retorna: a resposta da conversa
    """

    # Cria o index
    cria_vetor(leitura)

    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    # Carrega o db vetor
    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

    # Verifica a similaridade com a perqunta e index do vetor
    docs = new_db.similarity_search(user_question)

    # Cria o prompt
    chain = prompt_chain()

    # Faz a resposta
    response = chain(
        {"input_documents": docs, "question": user_question}, return_only_outputs=True
    )
   # Retorna a resposta
    return response['output_text']


###############-- PDF e Texto --#################
with st.form("form_texto"):
  arquivo = st.file_uploader("Escolha um PDF", accept_multiple_files=False,
                              type=['pdf'],
                             help="Utilize PDF para extrair o texto ou digita o Texto")
  leitura = ""
  if arquivo is not None:
    leitura = get_pdf(arquivo)

  texto = st.text_input('Texto 👇', leitura,key="entrada")

  selecao = st.radio(
      "Ações",
      ["Gerar Conteúdo", "Traduzir do Inglês para o Português", "Traduzir do Português para o Inglês","Resumir"],
      index=None,
      key="selecionar"
  )

  enviar = st.form_submit_button("Enviar!")

  if enviar:
    if ((texto is not None) and (len(texto) > 0)):
      if selecao == "Gerar Conteúdo":
        st.write("Resultado:", get_conteudo(texto))

      if selecao == "Traduzir do Inglês para o Português":
        st.write("Resultado:", get_conteudo(f"Traduzir do Inglês para o Português: {texto}"))

      if selecao == "Traduzir do Português para o Inglês":
       st.write("Resultado:", get_conteudo(f"Traduzir do Português para o Inglês: {texto}"))

      if selecao == "Resumir":
        st.write("Resultado:", get_conteudo(f"Resumir: {texto}"))

###############-- PDF e Texto--#################

###############--IMAGEM--#################
with st.form("form_imagem"):
  uploaded_imagem= st.file_uploader("Escolha uma Imagem", accept_multiple_files=False,
                                  type=['jpg', 'png'],
                                    help="Descreve uma imagem e verifica se é fake!"
                                    )
  if uploaded_imagem is not None:
      image = Image.open(uploaded_imagem)

      st.image(image, caption=f'Imagem Carregada: {uploaded_imagem.name}', use_column_width=True)
      bytes_data = uploaded_imagem.getvalue()

  generate = st.form_submit_button("Descrever!")

  if generate:
    if (uploaded_imagem is not None):
      model = genai.GenerativeModel('gemini-pro-vision')
      response = model.generate_content(
          glm.Content(
              parts = [
                  glm.Part(text="Descreva a imagem e verifica se a imagem é fake"),
                  glm.Part(
                      inline_data=glm.Blob(
                          mime_type='image/jpeg',
                          data=bytes_data
                      )
                  ),
              ],
          ),
          stream=True)

      response.resolve()

      st.write(response.text)

###############--IMAGEM--#################

###############-- Converse com PDF --#################
with st.form("form_conversa"):
  arquivo = st.file_uploader("Escolha um PDF", accept_multiple_files=False,
                              type=['pdf'],
                             help="Converse com seu PDF utizando embedding e Vector Store")

  if arquivo is not None:
    leitura = get_pdf(arquivo)
    st.write("Informação do PDF:", leitura)

  texto = st.text_input('Converse com o PDF 👇')

  conversar = st.form_submit_button("Conversar!")

  if conversar:
    if ((texto is not None) and (len(texto) > 0)):
      st.write("Resultado:", conversando(texto,leitura))

###############-- Converse com PDF --#################


st.button("Reset", type="primary", on_click=clear_form)


Overwriting app.py


In [32]:
# Para rodar diretamente do COLAB
# Será criado uma senha para usar na url.Exemplo:34.23.217.210
!npm install localtunnel
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com
!npx localtunnel --port 8501

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.445s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details
35.221.182.227
npx: installed 22 in 1.477s
your url is: https://ready-states-train.loca.lt
^C
